In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
#call open AI
openai = OpenAI()


In [4]:
# A class to represent a Webpage

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:
    def __init__(self, url):
        """
        Create this Website object for a Finance latest news
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        self.title = soup.title.string if soup.title else "No title found"
        
        # Find news headlines and content 
        news_data = []
        
        # Try different selectors 
        news_items = soup.find_all('h3') + soup.find_all('h2')
        
        for item in news_items:
            headline = item.get_text(strip=True)
            if headline and len(headline) > 20:  # Filter out short/empty text
                # Try to find content near the headline
                content = ""
                parent = item.find_parent()
                if parent:
                    # Look for paragraph or summary text
                    summary = parent.find('p')
                    if summary:
                        content = summary.get_text(strip=True)[:300] + "..."
                
                news_data.append({'headline': headline, 'content': content})
        
        # Create the text content
        self.text = "Latest financial news headlines:\n\n"
        
        # Get top 5 headlines with content
        for i, news in enumerate(news_data[:10], 1):
            self.text += f"{i}. {news['headline']}\n"
            if news['content']:
                self.text += f"   Summary: {news['content']}\n"
            self.text += "\n"
        
        if not news_data:
            self.text = "No headlines found. Yahoo Finance structure may have changed."


In [5]:
website = Website("https://finance.yahoo.com/topic/latest-news/")

print("Title:", website.title)
print("Top News:")
print(website.text)

Title: Yahoo Finance - Stock Market Live, Quotes, Business & Finance News
Top News:
Latest financial news headlines:

1. US Risks Losing ‘Reliable Investment’ Status, Allianz GI Manager Says
   Summary: (Bloomberg) -- Inside one of Europe’s biggest asset managers, there’s growing concern that Republican efforts to gut legislation supporting key industries such as clean energy may result in the US losing its status as a destination for investor capital.Most Read from BloombergNY Private School Plead...

2. Why Intempus thinks robots should have a human physiological state
   Summary: Teddy Warner, 19, has always been interested in robotics. His family was in the industry, and he says he "grew up" working in a machinist shop while in high school. Now Warner is building a robotics company of his own, Intempus, that looks to make robots a bit more human. Intempus is building tech t...

3. Last 24 hours: TechCrunch Disrupt 2025 Early Bird Deals will fly away after today
   Summary: Just 24 

In [22]:
# Define our system prompt
system_prompt = """You are a veteran stock market and finance expert with 50+ years of experience helping investors make safe, steady gains. Your audience is beginners with small amounts to invest (around $100). 

**Response Format:**
1. Start with "The News Snapshot:" - Write 3-4 lines summarizing the key financial developments from the provided headlines and summaries, showing you understand the current market situation, start the write up for this with today in the news we see that...

2. Give specific stock advice based on the news:
  - What to avoid and why
  - 2-3 specific stock recommendations with ticker symbols
  - Focus only on safe, dividend-paying stocks or clear beneficiaries from the news

3. End with "The big picture:" - One sentence explaining the overall market condition

4. Close with "Your game plan:" - Simple, actionable advice for their $100 to show how to split it

**Tone & Style:**
- Talk like a knowledgeable but friendly Wall Street professional advising a beginner
- Keep it under 200 words total
- Use simple language, no complex jargon
- Be direct and practical
- Focus on capital preservation over quick gains
- Always relate advice directly to the news headlines provided

**Key Rules:**
- Only recommend established, safe stocks
- Always explain WHY based on the news
- No speculative or meme stocks
- Emphasize learning over quick profits"""

In [23]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a provide your investment advice for a beginner with $100. \
Because it includes finance news or trend, let the advice be based on these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [24]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [25]:
def get_advice(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [10]:
get_advice("https://finance.yahoo.com/topic/latest-news/")

'**The News Snapshot:** Recent headlines reveal rising treasury yields and concerns over the US losing its "reliable investment" status, stoking fears of market uncertainty. Amidst this backdrop, investors may want to focus on stable, dividend-paying stocks that can weather the storm and provide consistent returns.\n\n**Stock Advice:**\n- **Avoid speculative tech stocks** like Nvidia, which has recently shown volatility and uncertainty in earnings, leading to a potential loss of investor confidence.\n- **Recommendation #1: Johnson & Johnson (JNJ)** – A well-established healthcare company that pays a reliable dividend, making it a safe bet in uncertain times.\n- **Recommendation #2: Procter & Gamble (PG)** – Known for its strong brand portfolio and consistent dividend payouts, PG offers stability and resilience against market fluctuations.\n- **Recommendation #3: Coca-Cola (KO)** – With a history of dividend increases, Coca-Cola remains a staple in many portfolios, providing that defens

In [26]:
def display_fin_advice(url):
    advice_content = get_advice(url)  
    display(Markdown(advice_content))

In [27]:
display_fin_advice("https://finance.yahoo.com/topic/latest-news/")

The News Snapshot: Today in the news, we see that concerns are rising around the US potentially losing its appeal as a reliable investment destination due to political actions, particularly in clean energy. Rising Treasury yields and fiscal uncertainty, stemming from tax policies, are causing unease in the markets. Generally, investors are on alert due to potential repercussions for sectors reliant on government support and tax reform.

Specific Stock Advice:
- I advise avoiding high-growth tech stocks like **Nvidia (NVDA)** for now, as their recent earnings show volatility and uncertainty. 
- Instead, consider established dividend-paying stocks like **Johnson & Johnson (JNJ)** and **Procter & Gamble (PG)**. Both companies are less sensitive to political changes and provide steady dividends, making them safer bets during turbulent times.
- Another option is **3M Company (MMM)**, which has a strong history of dividend payments and benefits from potential manufacturing boosts tied to new legislation.

The big picture: The market is navigating through uncertainties, particularly around fiscal policy and investment confidence.

Your game plan: Split your $100 into three parts: $40 in Johnson & Johnson, $40 in Procter & Gamble, and keep $20 in cash for future opportunities or to cover transaction fees. This balanced approach aims for safety and steady growth.